In [1]:
import numpy as np
import itertools
import nltk
import os
import shutil
import io

import torch
import torch.nn as nn

In [5]:
data = torch.load("language_rnn_checkpoint.pth.tar")

In [6]:
data

{'Epoch': 75,
 'Test_acc': [0.1921580000000001,
  0.21266360000000015,
  0.22426240000000003,
  0.2313943999999998,
  0.2365840000000001,
  0.24104359999999997,
  0.24453000000000008,
  0.24721039999999997,
  0.24905639999999998,
  0.25074120000000005,
  0.2524464,
  0.2540400000000001,
  0.2551092,
  0.25573520000000005,
  0.2568604,
  0.2578596000000001,
  0.2586648000000001,
  0.2594952,
  0.26005880000000015,
  0.2606432000000001,
  0.26106039999999997,
  0.2611347999999999,
  0.2616292,
  0.2624212000000001,
  0.26322280000000003,
  0.2638548,
  0.26331400000000005,
  0.264058,
  0.2641699999999999,
  0.2645088000000001,
  0.2646455999999999,
  0.26498160000000004,
  0.2658479999999999,
  0.26528320000000005,
  0.26557119999999995,
  0.2660075999999999,
  0.26650439999999986,
  0.2665235999999999,
  0.26631960000000005,
  0.26671279999999986,
  0.26678320000000016,
  0.2675772,
  0.2666783999999999,
  0.26702919999999986,
  0.2674476000000001,
  0.26729640000000005,
  0.2673432000

In [2]:
embedding = nn.Embedding(80,5)
x = torch.LongTensor(np.random.randn(500,80))

In [7]:
y = x[:,1:].contiguous().view(-1)
x.size(), y.size()

(torch.Size([500, 80]), torch.Size([39500]))

In [4]:
embed = embedding(x)

RuntimeError: index out of range at c:\programdata\miniconda3\conda-bld\pytorch_1532505617613\work\aten\src\th\generic/THTensorMath.cpp:352

In [24]:
checkpoint = os.listdir("Part 2 Output")
param_group = []
cpt = torch.load(os.path.join("Part 2 Output",checkpoint[15]))
#np.around(cpt["Train_loss"],decimals=6), np.around(cpt["Train_acc"],3)
np.around(cpt["Test_acc"],decimals=4)

array([0.6024, 0.6548, 0.6765, 0.6843, 0.686 , 0.689 , 0.6882, 0.6898,
       0.69  , 0.6888, 0.6887, 0.689 , 0.689 , 0.6889, 0.689 , 0.6891,
       0.6888, 0.689 , 0.6891, 0.6891])

In [41]:
checkpoint = os.listdir("Part 1 Output")
param_group = []
cpt = torch.load(os.path.join("Part 1 Output",checkpoint[1]))
#np.around(cpt["Train_loss"],decimals=6), np.around(cpt["Train_acc"],3)
np.around(cpt["Test_acc"],decimals=4)

array([0.8154, 0.8729, 0.8789, 0.8927, 0.902 , 0.9109, 0.9117, 0.9125,
       0.9155, 0.9177, 0.9175, 0.9161, 0.9171, 0.9205, 0.9158, 0.9215,
       0.9233, 0.922 , 0.9214, 0.9216])

In [5]:
import numpy as np
x_train = [["it", "is", "raining", "now", "."], ["i", "am", "taking", "class", "."],
           ["how", "is", "it", "going", "?"], ["do", "you", "like", "food", "?"]]
token_ids = np.asarray([[token for token in x] for x in x_train])
token_ids.shape

(4, 5)

In [16]:
## word_to_id and id_to_word. associate an id to every unique token in the training data
## Hence we can build our own vocabulary
all_tokens = itertools.chain.from_iterable(x_train)
word_to_id = {token:idx for idx,token in enumerate(set(all_tokens))}

all_tokens = itertools.chain.from_iterable(x_train)
id_to_word = [token for idx, token in enumerate(set(all_tokens))]
id_to_word = np.asarray(id_to_word)
id_to_word

array(['am', 'how', '.', '?', 'is', 'taking', 'like', 'class', 'you',
       'the', 'going', 'a', 'it', 'raining', 'do', 'i', 'food'],
      dtype='<U7')

In [19]:
word_to_id = {token:idx for idx,token in enumerate(set(x_train))}

id_to_word = [token for idx, token in enumerate(set(x_train))]
id_to_word = np.asarray(id_to_word)
id_to_word

TypeError: unhashable type: 'list'

In [20]:
## let's sort the indices by word frequency instead of random
x_train_token_ids = [[word_to_id[token] for token in x] for x in x_train]
count = np.zeros(id_to_word.shape)
for x in x_train_token_ids:
    for token in x:
        count[token] += 1
indices = np.argsort(-count)
id_to_word = id_to_word[indices]
count = count[indices]
id_to_word, count

(array(['is', 'it', 'am', 'like', '.', 'how', 'taking', 'class', 'i',
        'going', 'the', 'a', 'raining', 'do', '?', 'you', 'food'],
       dtype='<U7'),
 array([2., 2., 2., 2., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))

In [22]:
## recreate word_to_id based on sorted list
word_to_id = {token: idx for idx, token in enumerate(id_to_word)}

## assign -1 if token doesn't appear in our dictionary
## add +1 to all token ids, we went to reserve id=0 for an unknown token
x_train_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_train]
#x_test_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_test]

In [24]:
x_train_token_ids

[[2, 1, 13, 5],
 [9, 3, 7, 12, 8, 5],
 [6, 1, 2, 10, 15],
 [14, 16, 4, 11, 17, 15]]

In [25]:
## save dictionary
np.save('exp_dictionary.npy',np.asarray(id_to_word))

## save training data to single text file
with io.open('exp_imdb_train.txt','w',encoding='utf-8') as f:
    for tokens in x_train_token_ids:
        for token in tokens:
            f.write("%i " % token)
        f.write("\n")

In [ ]:
## get all of the training reviews (including unlabeled reviews)
train_directory = '/projects/training/bauh/NLP/aclImdb/train/'

pos_filenames = os.listdir(train_directory + 'pos/')
neg_filenames = os.listdir(train_directory + 'neg/')
unsup_filenames = os.listdir(train_directory + 'unsup/')

pos_filenames = [train_directory+'pos/'+filename for filename in pos_filenames]
neg_filenames = [train_directory+'neg/'+filename for filename in neg_filenames]
unsup_filenames = [train_directory+'unsup/'+filename for filename in unsup_filenames]

filenames = pos_filenames + neg_filenames + unsup_filenames

x_train = []
for filename in filenames:
    with io.open(filename,'r',encoding='utf-8') as f:
        line = f.readlines()[0]
    line = line.replace('<br />',' ')
    line = line.replace('\x96',' ')
    line = nltk.word_tokenize(line)
    line = [w.lower() for w in line]
    x_train.append(line)

In [ ]:
## get all of the test reviews
test_directory = '/projects/training/bauh/NLP/aclImdb/test/'

pos_filenames = os.listdir(test_directory + 'pos/')
neg_filenames = os.listdir(test_directory + 'neg/')

pos_filenames = [test_directory+'pos/'+filename for filename in pos_filenames]
neg_filenames = [test_directory+'neg/'+filename for filename in neg_filenames]

filenames = pos_filenames+neg_filenames

x_test = []
for filename in filenames:
    with io.open(filename,'r',encoding='utf-8') as f:
        line = f.readlines()[0]
    line = line.replace('<br />',' ')
    line = line.replace('\x96',' ')
    line = nltk.word_tokenize(line)
    line = [w.lower() for w in line]
    x_test.append(line)

In [ ]:
## word_to_id and id_to_word. associate an id to every unique token in the training data
## Hence we can build our own vocabulary
all_tokens = itertools.chain.from_iterable(x_train)
id_to_word = [token for idx, token in enumerate(set(all_tokens))]
id_to_word = np.asarray(id_to_word)

In [ ]:
## let's sort the indices by word frequency instead of random
x_train_token_ids = [[word_to_id[token] for token in x] for x in x_train]
count = np.zeros(id_to_word.shape)
for x in x_train_token_ids:
    for token in x:
        count[token] += 1
indices = np.argsort(-count)
id_to_word = id_to_word[indices]
count = count[indices]

In [ ]:
## recreate word_to_id based on sorted list
word_to_id = {token: idx for idx, token in enumerate(id_to_word)}

## assign -1 if token doesn't appear in our dictionary
## add +1 to all token ids, we went to reserve id=0 for an unknown token
x_train_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_train]
x_test_token_ids = [[word_to_id.get(token,-1)+1 for token in x] for x in x_test]